In [1]:
# Dependencies 
import pandas as pd
import numpy as np

In [2]:
# File to load 
census_df = pd.read_csv ("census_pop_density.csv")

In [3]:
import unicodedata
currentFile = open("NCHS.csv", "rt",encoding="utf-8")

In [4]:
NCHS_df = pd.read_csv (currentFile)

In [5]:
# Cleaning Census Data
census_df.head()
for col in census_df.columns:
    print(col)

OBJECTID
COUNTYNS
GEOID
ALAND
AWATER
NAME
State
B25010_001E
B25010_001M
B25010_002E
B25010_002M
B25010_003E
B25010_003M
B01001_001E
B01001_001M
B01001_calc_PopDensity
created_user
created_date
last_edited_user
last_edited_date
B01001_calc_PopDensityM
SHAPE_Length
SHAPE_Area


In [6]:
census_df=census_df.drop(columns=["OBJECTID", "COUNTYNS", "ALAND", "AWATER","B25010_002E",
"B25010_002M", "B25010_003E", "B25010_003M","B01001_001M", "created_user", "created_date",
"last_edited_user", "last_edited_date", "B01001_calc_PopDensityM", "SHAPE_Length","SHAPE_Area","B25010_001M"])

In [7]:
census_df=census_df.rename(columns={"GEOID":"fips_code","NAME": "county_name","State":"state","B25010_001E":"avg_household","B01001_calc_PopDensity":"pop_dens","B01001_001E":"total_pop" })
census_df.head()

,fips_code,county_name,state,avg_household,total_pop,pop_dens
0,1001,Autauga County,Alabama,2.59,55200,35.853419
1,1003,Baldwin County,Alabama,2.61,208107,50.541504
2,1005,Barbour County,Alabama,2.49,25782,11.247981
3,1007,Bibb County,Alabama,2.99,22527,13.973114
4,1009,Blount County,Alabama,2.77,57645,34.515816


In [8]:
# Cleaning NHCS Data
NCHS_df.head()

,FIPS code,State Abr.,County name,County 2012 pop,2013 code,2006 code,1990-based code
0,1001,AL,Autauga County,55514,3,3,3
1,1003,AL,Baldwin County,190790,4,5,3
2,1005,AL,Barbour County,27201,6,5,5
3,1007,AL,Bibb County,22597,2,2,6
4,1009,AL,Blount County,57826,2,2,3


In [9]:
NCHS_df=NCHS_df.drop(columns={"County 2012 pop","2006 code", "1990-based code"})

In [10]:

NCHS_df.head()

,FIPS code,State Abr.,County name,2013 code
0,1001,AL,Autauga County,3
1,1003,AL,Baldwin County,4
2,1005,AL,Barbour County,6
3,1007,AL,Bibb County,2
4,1009,AL,Blount County,2


In [11]:
NCHS_df=NCHS_df.rename(columns={"FIPS code":"fips_code","State Abr.":"state_abr","County name":"county_name","2013 code":"nchs_code_2012"})
NCHS_df.head()

,fips_code,state_abr,county_name,nchs_code_2012
0,1001,AL,Autauga County,3
1,1003,AL,Baldwin County,4
2,1005,AL,Barbour County,6
3,1007,AL,Bibb County,2
4,1009,AL,Blount County,2


In [12]:
# Combining the dataframes
county_pop_df=pd.merge(census_df,NCHS_df,how="inner",on=["fips_code","county_name"])

In [13]:
county_pop_df.head()

,fips_code,county_name,state,avg_household,total_pop,pop_dens,state_abr,nchs_code_2012
0,1001,Autauga County,Alabama,2.59,55200,35.853419,AL,3
1,1003,Baldwin County,Alabama,2.61,208107,50.541504,AL,4
2,1005,Barbour County,Alabama,2.49,25782,11.247981,AL,6
3,1007,Bibb County,Alabama,2.99,22527,13.973114,AL,2
4,1009,Blount County,Alabama,2.77,57645,34.515816,AL,2


In [14]:
county_pop_df=county_pop_df.loc[(county_pop_df["nchs_code_2012"]<=2)&(county_pop_df["pop_dens"]>1000)]
county_pop_df.drop_duplicates()
county_pop_df.to_csv('county_pop.csv')

In [15]:
# Dense County Data
dense_state_county = [
    ("Maryland","Baltimore city"),
    ("New Jersey","Essex"),
    ("Illinois","Cook"),
    ("New Jersey","Union"),
    ("Virginia","Norfolk city"),
    ("New York","Nassau"),
    ("Virginia","Harrisonburg city")]

dense_county_pop_df = pd.DataFrame()
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)

for state, county in dense_state_county:
    dense_county_pop_df = county_pop_df[(county_pop_df["state"] == state) & (county_pop_df["county_name"] == county)].reset_index().drop(["index"],1)
dense_county_pop_df.head()

,fips_code,county_name,state,avg_household,total_pop,pop_dens,state_abr,nchs_code_2012
